# Практическое занятие 8
# Компьютерный практикум по алгебре на Python
## Численное решение систем линейных алгебраических уравнений (СЛАУ) с numpy.

https://numpy.org/doc/stable/reference/routines.linalg.html

In [1]:
import numpy as np
from numpy import linalg

#Представление матриц в numpy.

!!! Сейчас матрицы в numpy рекомендуется представлять в виде структуры данных "2d numpy.array object", а не a "numpy.matrix object", даже для задач линейной алгебры.

2d numpy.array object это вложенный (двумерный) массив, его элементы (в отличие от сисков list) данные одного типа, т.е. все элементы числа int или все комплексные числа и т.п.

In [2]:
my_matr = np.array([[1, 2, 3], [4, 5, 6]])
my_matr

array([[1, 2, 3],
       [4, 5, 6]])

Если конструктору numpy.array передать список, содержащий разные типы чисел, то все числа преобразуются к наиболее общему типу. Например, если в списке есть int и float, то все станут float, а если еще есть комплексные числа, то все будут преобразованы в комплексные числа.

In [3]:
my_matr1 = np.array([[1, 2.5, 3], [4, 5, 6]])
my_matr2 = np.array([[1, 2 + 1j, 3], [4, 5, 6]])
print(my_matr1, my_matr2, sep='\n')

[[1.  2.5 3. ]
 [4.  5.  6. ]]
[[1.+0.j 2.+1.j 3.+0.j]
 [4.+0.j 5.+0.j 6.+0.j]]


Для некоторых стандартных видов матриц есть конструкторы, например 

для **единичной** матрицы numpy.identity(n, dtype=None, *, like=None) и numpy.eye(N, M=None, k=0, dtype=<class 'float'>, order='C', *, like=None)

для **матрицы из нулей** numpy.zeros(shape, dtype=float, order='C', *, like=None)

In [4]:
print(f"""np.identity(3):\n{np.identity(3)},
np.eye(3, 4):\n{np.eye(3, 4)},
np.eye(3, 4, dtype=int):\n{np.eye(3, 4, dtype=int)},
np.zeros((2, 3)):\n{np.zeros((2, 3))},
np.zeros((2, 3), dtype=complex):\n{np.zeros((2, 3), dtype=complex)}.""", sep='\n')

np.identity(3):
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]],
np.eye(3, 4):
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]],
np.eye(3, 4, dtype=int):
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]],
np.zeros((2, 3)):
[[0. 0. 0.]
 [0. 0. 0.]],
np.zeros((2, 3), dtype=complex):
[[0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j]].


## numpy.linalg.solve

linalg.solve это решатель СЛАУ из N уравнений от N переменных.Возможное число решений одно, иначе выдается ошибка. 

### Пример 1. Совместная СЛАУ с единственным решением. 


In [5]:
A = np.array([[2, 3, -1], [3, -2, 1], [1, 1, -1]])
b = np.array([5, 2, 0])
X = linalg.solve(A, b)
print(f'Решение СЛАУ: {X}')

Решение СЛАУ: [1. 2. 3.]


Как выполнить проверку? В случае отсутствия ошибок округлений при вычислениях должно выполняться $AX = b$.

Для умножения матриц (т.е. array) используется оператор @ или метод matmul (то же, что и @):

In [6]:
A @ X == b

array([ True,  True, False])

Как видим, матрицы-столбцы левой и правой частей совпадают только в одной координате. Почему? По причине округлений при вычислениях. Посмотрим, насколько сильно отличаются левая и правая части:

In [7]:
print(f'A @ X = {A @ X},\nb = {b}')

A @ X = [5.0000000e+00 2.0000000e+00 4.4408921e-16],
b = [5 2 0]


Поскольку в жизни при использовании приближенных вычислений всегда результат подстановки решения будет несколько отличаться от правой части, то сравнение проводится лишь с некоторой точностью с помощью numpy.allclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)

https://numpy.org/devdocs/reference/generated/numpy.allclose.html

numpy.allclose() сравнивает поэлементно первый и второй свои аргументы и возвращает True, если они все отличаются друг от друга не более чем на допустимую величину, определяемую параметрами rtol (относительная погрешность) и atol (абсолютная погрешность).

При сравнении друх чисел $a$ и $b$ вычисляется величина $|a - b|$  и сравнивается с допустимой погрешностью $atol + rtol * |b|$. Если выполняется $|a - b|\le atol + rtol * |b|$, то считается, что $a$ и $b$ приближенно равны.

In [8]:
np.allclose(A @ X, b)

True

### Пример 2. Несовместная СЛАУ
$$
\left\{
\begin{matrix}
2x + 3y - z = 5\\
3x - 2y + z = 2\\
5x + y = 0
\end{matrix}
\right.
$$

В случае этой несовместной СЛАУ linalg.solve выдает ошибку "Singular matrix". Чтобы программа не завершалась ошибкой, будем вычислять определитель левой части (если матрица квадратная) или ранг (в общем случае).

In [19]:
A = np.array([[2, 3, -1], [3, -2, 1], [5, 1, 0], [5, 1, 0]])
# b = np.array([5, 2, 0])
# print(f'Определитель |А| = {linalg.det(A)}, ранг rg(А) = {linalg.matrix_rank(A)}')

A

array([[ 2,  3, -1],
       [ 3, -2,  1],
       [ 5,  1,  0],
       [ 5,  1,  0]])

Для проверки СЛАУ на совместность по теореме Кронекера-Капелли нужно определить ранг расширенной матрицы СЛАУ, а для этого надо эту матрицу составить. Для соединения двух матриц в одну в numpy есть много функций (concatenate, stack, hstack, vstack и т.п), но в нашем случае, когда нужно к двумерному массиву присоединить одномерный столбец, лучше подойдет column_stack().

In [10]:
Ab = np.column_stack((A, b))
print(f'Ab:\n{Ab}\nранг Ab rg(Аb) = {linalg.matrix_rank(Ab)}')

Ab:
[[ 2  3 -1  5]
 [ 3 -2  1  2]
 [ 5  1  0  0]]
ранг Ab rg(Аb) = 3


Видим, что ранг расширенной матрицы больше, чем ранг левой части, следовательно, СЛАУ несовместна.

### Пример 3. Недоопределенная СЛАУ
$$
\left\{
\begin{matrix}
2x + 3y - z = 5\\
3x - 2y + z = 2\\
5x + y = 7
\end{matrix}
\right.
$$

In [11]:
A = np.array([[2, 3, -1], [3, -2, 1], [5, 1, 0]])
b = np.array([5, 2, 7])
Ab = np.column_stack((A, b))
print(f'ранг A rg(А) = {linalg.matrix_rank(A)}, ранг Ab rg(Аb) = {linalg.matrix_rank(Ab)}')

ранг A rg(А) = 2, ранг Ab rg(Аb) = 2


С помощью linalg.solve() такую СЛАУ решить нельзя, поскольку матрица левой части неполного ранга.

### Выделение строки, столбца и подматрицы в np.array
Для выделения части матрицы будем использовать диапазоны (срезы, slice)

Напомним, что обращение к элементу матрицы осуществляется указанием в квадратных скобках после имени матрицы номера строки и номера столбца через запятую,

например, $А[2, 3]$.

Если нужно выделить элементы строки, стоящие в столбцах с 3 до 5 (не включая 5!), то вместо номера столбца указываем диапазон (срез) 3:5,
двоеточие обозначает, что берутся и все промежуточные индексы.

**!!!ВАЖНО!!!**

В диапазонах не учитывается последнее значение,
так срез 3:5 не включает элемент с номером 5!

In [12]:
Qmatr = np.array([[i * j for i in range(1, 9)] for j in range(2, 8)])
Qmatr

array([[ 2,  4,  6,  8, 10, 12, 14, 16],
       [ 3,  6,  9, 12, 15, 18, 21, 24],
       [ 4,  8, 12, 16, 20, 24, 28, 32],
       [ 5, 10, 15, 20, 25, 30, 35, 40],
       [ 6, 12, 18, 24, 30, 36, 42, 48],
       [ 7, 14, 21, 28, 35, 42, 49, 56]])

Выделим столбец с номером 1 в матрице Qmatr

In [13]:
Qmatr[:, 1]

array([ 4,  6,  8, 10, 12, 14])

Выделим строку с номером 0 в матрице Qmatr

In [14]:
Qmatr[0, :]

array([ 2,  4,  6,  8, 10, 12, 14, 16])

Допишем снизу к матрице Qmatr ее строку номер 1

In [15]:
np.row_stack((Qmatr, Qmatr[1, :]))

array([[ 2,  4,  6,  8, 10, 12, 14, 16],
       [ 3,  6,  9, 12, 15, 18, 21, 24],
       [ 4,  8, 12, 16, 20, 24, 28, 32],
       [ 5, 10, 15, 20, 25, 30, 35, 40],
       [ 6, 12, 18, 24, 30, 36, 42, 48],
       [ 7, 14, 21, 28, 35, 42, 49, 56],
       [ 3,  6,  9, 12, 15, 18, 21, 24]])

Допишем справа к матрице $A$ ее столбец номер 0

In [16]:
np.column_stack((Qmatr, Qmatr[:, 0]))

array([[ 2,  4,  6,  8, 10, 12, 14, 16,  2],
       [ 3,  6,  9, 12, 15, 18, 21, 24,  3],
       [ 4,  8, 12, 16, 20, 24, 28, 32,  4],
       [ 5, 10, 15, 20, 25, 30, 35, 40,  5],
       [ 6, 12, 18, 24, 30, 36, 42, 48,  6],
       [ 7, 14, 21, 28, 35, 42, 49, 56,  7]])

**Заметим, что np.row_stack и np.column_stack возвращают результат, не меняя саму матрицу.**

### Транспонирование
Транспонируем Qmatr с помощью transpose().

In [17]:
np.transpose(Qmatr)

array([[ 2,  3,  4,  5,  6,  7],
       [ 4,  6,  8, 10, 12, 14],
       [ 6,  9, 12, 15, 18, 21],
       [ 8, 12, 16, 20, 24, 28],
       [10, 15, 20, 25, 30, 35],
       [12, 18, 24, 30, 36, 42],
       [14, 21, 28, 35, 42, 49],
       [16, 24, 32, 40, 48, 56]])